In [1]:
# imports
import os
import re
import pandas as pd
import json
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, concat, monotonically_increasing_id, length, udf, when, lower
from pyspark.sql.types import IntegerType, StringType

from confluent_kafka import Producer

# setup paths - detect project root
cwd = os.getcwd()
if 'notebooks' in cwd:
    PROJECT_ROOT = os.path.dirname(os.path.dirname(cwd))  # TWO levels up
else:
    PROJECT_ROOT = cwd
sys.path.insert(0, PROJECT_ROOT)

# kafka config
KAFKA_BROKER = os.environ.get('KAFKA_BROKER', 'kafka-broker:29092')

# sampling config
TARGET_SAMPLE_SIZE = 150000  # 150K jobs for balanced training with 7K CVs
MIN_PER_DOMAIN = 1000        # minimum samples per ISCO domain

print('imports loaded')
print(f'project root: {PROJECT_ROOT}')
print(f'kafka broker: {KAFKA_BROKER}')
print(f'target sample size: {TARGET_SAMPLE_SIZE:,}')

imports loaded
project root: /home/developer/project
kafka broker: kafka-broker:29092
target sample size: 150,000


In [2]:
spark = SparkSession.builder \
    .appName('FinalEmbeddingOutput') \
    .config('spark.driver.memory', '4g') \
    .getOrCreate()

print(f'spark started: {spark.version}')

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/27 01:28:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark started: 4.1.1


In [3]:
input_path = os.path.join(PROJECT_ROOT, 'ingest_job_postings', 'output', 'unified_job_postings', 'unified_jobs.parquet')
jobs_df = spark.read.parquet(input_path)
total_jobs = jobs_df.count()

print(f'loaded {total_jobs:,} jobs')
print(f'columns: {jobs_df.columns}')
print('sample:')
jobs_df.select('id', 'job_title', 'embedding_text').show(3, truncate=60)

loaded 1,345,711 jobs
columns: ['id', 'job_title', 'company', 'job_location', 'skills', 'seniority', 'embedding_text']
sample:
+------------------------------------+-----------------------------------------------------+------------------------------------------------------------+
|                                  id|                                            job_title|                                              embedding_text|
+------------------------------------+-----------------------------------------------------+------------------------------------------------------------+
|71af9d85-4b75-40d6-9835-38d7153df1d5|"Telephonic" Nurse Case Manager II at Elevance Health|Role of "Telephonic" Nurse Case Manager II at Elevance He...|
|e19c544b-810f-42b8-a861-b68bc9be0d71|"Telephonic" Nurse Case Manager II at Elevance Health|Role of "Telephonic" Nurse Case Manager II at Elevance He...|
|4f9d65ee-1255-429e-8fcd-127196cc1080|          $10,000 Sign on Bonus Laboratory Supervisor|Role of $10

In [4]:
# Fast ID assignment using monotonically_increasing_id()---> ~100x faster than zipWithIndex because it doesn't require shuffle
# IDs will have gaps (B0, B1, B8589934592...) should not be a problem for embeddings

def assign_sequential_ids_fast(df, prefix='B'):
    if 'id' in df.columns:
        df = df.withColumnRenamed('id', 'original_uuid')
    
    # monotonically_increasing_id, distributed and fast
    df = df.withColumn(
        'job_id',
        concat(lit(prefix), monotonically_increasing_id().cast('string'))
    )
    
    return df

print('fast ID assignment function defined (uses monotonically_increasing_id)')

fast ID assignment function defined (uses monotonically_increasing_id)


In [5]:
print('assigning B IDs (fast method)...')
jobs_with_ids = assign_sequential_ids_fast(jobs_df, prefix='B')

# count is now fast since we're not using RDD
job_count = jobs_with_ids.count()
print(f'assigned IDs to {job_count:,} jobs')

print('sample with IDs:')
jobs_with_ids.select('job_id', 'original_uuid', 'job_title').show(10, truncate=60)

assigning B IDs (fast method)...
assigned IDs to 1,345,711 jobs
sample with IDs:
+------+------------------------------------+------------------------------------------------------------+
|job_id|                       original_uuid|                                                   job_title|
+------+------------------------------------+------------------------------------------------------------+
|    B0|71af9d85-4b75-40d6-9835-38d7153df1d5|       "Telephonic" Nurse Case Manager II at Elevance Health|
|    B1|e19c544b-810f-42b8-a861-b68bc9be0d71|       "Telephonic" Nurse Case Manager II at Elevance Health|
|    B2|4f9d65ee-1255-429e-8fcd-127196cc1080|                 $10,000 Sign on Bonus Laboratory Supervisor|
|    B3|fb06029b-149b-4093-903a-f86a06e182a5|       $10K Bonus - In Patient RN, Dialysis Registered Nurse|
|    B4|83ba594c-6dd6-46e5-93f0-0646e2a1cbeb|                       $16/Hr Experienced Department Manager|
|    B5|b84d9d28-40f7-4086-a976-e11f2ac4c72c|$17.00 Anytime ava

In [6]:
# ISCO-08 classification patterns based on International Standard Classification of Occupations
#ref: https://www.ilo.org/public/english/bureau/stat/isco/isco08/

ISCO_PATTERNS = {
    1: ['manager', 'director', 'chief', 'head of', 'president', 'vp ', 'vice president', 
        'ceo', 'cfo', 'cto', 'coo', 'executive', 'supervisor', 'lead', 'principal'],
    2: ['engineer', 'developer', 'scientist', 'doctor', 'nurse', 'teacher', 'professor',
        'architect', 'designer', 'analyst', 'consultant', 'specialist', 'attorney', 
        'lawyer', 'accountant', 'pharmacist', 'therapist', 'physician'],
    3: ['technician', 'associate', 'assistant', 'coordinator', 'representative',
        'administrator', 'officer', 'agent', 'inspector', 'controller'],
    4: ['clerk', 'secretary', 'receptionist', 'cashier', 'teller', 'bookkeeper',
        'data entry', 'typist', 'filing'],
    5: ['sales', 'retail', 'customer service', 'waiter', 'waitress', 'bartender',
        'cook', 'chef', 'server', 'barista', 'store', 'shop'],
    6: ['farmer', 'agricultural', 'fisherman', 'forestry', 'gardener', 'rancher'],
    7: ['electrician', 'mechanic', 'plumber', 'carpenter', 'welder', 'machinist',
        'installer', 'repair', 'maintenance', 'hvac'],
    8: ['driver', 'operator', 'machine', 'assembler', 'production', 'manufacturing',
        'warehouse', 'forklift', 'truck'],
    9: ['cleaner', 'janitor', 'helper', 'laborer', 'packer', 'loader', 'dishwasher',
        'housekeeper', 'security guard'],
    0: ['military', 'army', 'navy', 'marine', 'air force', 'soldier']
}

ISCO_NAMES = {
    0: 'Military',
    1: 'Managers',
    2: 'Professionals', 
    3: 'Technicians',
    4: 'Clerical',
    5: 'Service/Sales',
    6: 'Agriculture',
    7: 'Craft/Trade',
    8: 'Operators',
    9: 'Elementary'
}

def classify_isco(job_title):
    #classify job title into ISCO-08 major group
    if not job_title:
        return 2  # default to Professionals
    
    title_lower = job_title.lower()
    
    # check each ISCO group's patterns
    for isco_code, patterns in ISCO_PATTERNS.items():
        for pattern in patterns:
            if pattern in title_lower:
                return isco_code
    
    # default to Professionals (most common in job datasets)
    return 2

# register as Spark UDF
classify_isco_udf = udf(classify_isco, IntegerType())

print('ISCO classification function defined, 10 ISCO major groups configured')

ISCO classification function defined, 10 ISCO major groups configured


In [7]:
# apply ISCO classification to all jobs
print('classifying jobs by ISCO domain')

jobs_with_isco = jobs_with_ids.withColumn(
    'isco_code',
    classify_isco_udf(col('job_title'))
)

# show distribution
print('ISCO distribution (before sampling):')
isco_counts = jobs_with_isco.groupBy('isco_code').count().orderBy('isco_code').collect()

total = sum(row['count'] for row in isco_counts)
for row in isco_counts:
    code = row['isco_code']
    count = row['count']
    pct = count / total * 100
    name = ISCO_NAMES.get(code, 'Unknown')
    print(f'  {code} - {name:15s}: {count:>10,} ({pct:5.1f}%)')

print(f'\ntotal jobs: {total:,}')

classifying jobs by ISCO domain
ISCO distribution (before sampling):


  0 - Military       :        639 (  0.0%)
  1 - Managers       :    492,215 ( 36.6%)
  2 - Professionals  :    691,783 ( 51.4%)
  3 - Technicians    :    100,870 (  7.5%)
  4 - Clerical       :      9,981 (  0.7%)
  5 - Service/Sales  :     25,491 (  1.9%)
  6 - Agriculture    :        157 (  0.0%)
  7 - Craft/Trade    :     10,200 (  0.8%)
  8 - Operators      :     11,525 (  0.9%)
  9 - Elementary     :      2,850 (  0.2%)

total jobs: 1,345,711


In [8]:
# calculate stratified sampling fractions, proportional sampling with minimum per domain

print(f'calculating stratified sampling fractions, target: {TARGET_SAMPLE_SIZE:,} jobs, minimum per domain: {MIN_PER_DOMAIN:,}')

# get counts per domain
domain_counts = {row['isco_code']: row['count'] for row in isco_counts}

# calculate how many to sample from each domain, proportional to size, but ensure minimum
sample_per_domain = {}
remaining_budget = TARGET_SAMPLE_SIZE

# first pass: allocate minimum to small domains
for code, count in domain_counts.items():
    if count < MIN_PER_DOMAIN:
        # take all if domain is smaller than minimum
        sample_per_domain[code] = count
        remaining_budget -= count
    elif count * (TARGET_SAMPLE_SIZE / total) < MIN_PER_DOMAIN:
        # domain is small, give it minimum
        sample_per_domain[code] = MIN_PER_DOMAIN
        remaining_budget -= MIN_PER_DOMAIN

# second pass: proportional allocation for remaining
remaining_domains = {k: v for k, v in domain_counts.items() if k not in sample_per_domain}
remaining_total = sum(remaining_domains.values())

for code, count in remaining_domains.items():
    proportion = count / remaining_total
    sample_per_domain[code] = int(proportion * remaining_budget)

# calculate fractions for Spark sampleBy
fractions = {}
for code, sample_count in sample_per_domain.items():
    domain_total = domain_counts[code]
    fraction = min(1.0, sample_count / domain_total * 1.1)  # 10% buffer for randomness
    fractions[code] = fraction

print('\nsampling fractions per domain:')
for code in sorted(fractions.keys()):
    name = ISCO_NAMES.get(code, 'Unknown')
    frac = fractions[code]
    target = sample_per_domain[code]
    print(f'  {code} - {name:15s}: {frac:.3f} (target ~{target:,})')

calculating stratified sampling fractions, target: 150,000 jobs, minimum per domain: 1,000

sampling fractions per domain:
  0 - Military       : 1.000 (target ~639)
  1 - Managers       : 0.121 (target ~54,355)
  2 - Professionals  : 0.121 (target ~76,393)
  3 - Technicians    : 0.121 (target ~11,139)
  4 - Clerical       : 0.121 (target ~1,102)
  5 - Service/Sales  : 0.121 (target ~2,814)
  6 - Agriculture    : 1.000 (target ~157)
  7 - Craft/Trade    : 0.121 (target ~1,126)
  8 - Operators      : 0.121 (target ~1,272)
  9 - Elementary     : 0.386 (target ~1,000)


In [9]:
# perform stratified sampling using Spark's sampleBy
print('performing stratified sampling')

sampled_jobs = jobs_with_isco.sampleBy('isco_code', fractions, seed=42)

# cache and count
sampled_jobs.cache()
sampled_count = sampled_jobs.count()

print(f'sampled {sampled_count:,} jobs (target: {TARGET_SAMPLE_SIZE:,}, sampling ratio: {sampled_count/total*100:.1f}% of original')

# show new distribution
print('\nISCO distribution (after sampling):')
sampled_isco = sampled_jobs.groupBy('isco_code').count().orderBy('isco_code').collect()

for row in sampled_isco:
    code = row['isco_code']
    count = row['count']
    pct = count / sampled_count * 100
    name = ISCO_NAMES.get(code, 'Unknown')
    orig = domain_counts.get(code, 0)
    print(f'  {code} - {name:15s}: {count:>8,} ({pct:5.1f}%) [from {orig:,}]')

performing stratified sampling


sampled 165,193 jobs (target: 150,000, sampling ratio: 12.3% of original

ISCO distribution (after sampling):
  0 - Military       :      639 (  0.4%) [from 639]
  1 - Managers       :   59,897 ( 36.3%) [from 492,215]
  2 - Professionals  :   84,320 ( 51.0%) [from 691,783]
  3 - Technicians    :   12,146 (  7.4%) [from 100,870]
  4 - Clerical       :    1,220 (  0.7%) [from 9,981]
  5 - Service/Sales  :    3,051 (  1.8%) [from 25,491]
  6 - Agriculture    :      157 (  0.1%) [from 157]
  7 - Craft/Trade    :    1,255 (  0.8%) [from 10,200]
  8 - Operators      :    1,418 (  0.9%) [from 11,525]
  9 - Elementary     :    1,090 (  0.7%) [from 2,850]


In [10]:
# add passage prefix to sampled jobs
sampled_with_prefix = sampled_jobs.withColumn(
    'embedding_text',
    concat(lit('passage: '), col('embedding_text'))
)

print('added passage prefix to sampled jobs')
print('sample:')
sampled_with_prefix.select('job_id', 'isco_code', 'embedding_text').show(5, truncate=70)

added passage prefix to sampled jobs
sample:
+------+---------+----------------------------------------------------------------------+
|job_id|isco_code|                                                        embedding_text|
+------+---------+----------------------------------------------------------------------+
|    B7|        1|passage: Role of $18.00 Assistant Manager at McDonald's in Whitevil...|
|   B16|        1|passage: Role of $4,500 Sign on Bonus - MDS Coordinator at The Good...|
|   B18|        2|passage: Role of $45/hr - School RN at Maxim Healthcare Staffing in...|
|   B50|        1|passage: Role of (CAN) Department Manager -FR at Walmart Canada in ...|
|   B58|        1|passage: Role of (CAN) Department Manager at Walmart Canada in Kitc...|
+------+---------+----------------------------------------------------------------------+
only showing top 5 rows


In [11]:
# select final columns for embedding
final_output = sampled_with_prefix.select('job_id', 'embedding_text', 'isco_code')

print('final output columns:')
print(final_output.columns)
print(f'total records: {sampled_count:,}')
print('\nsample:')
final_output.show(10, truncate=70)

final output columns:
['job_id', 'embedding_text', 'isco_code']
total records: 165,193

sample:
+------+----------------------------------------------------------------------+---------+
|job_id|                                                        embedding_text|isco_code|
+------+----------------------------------------------------------------------+---------+
|    B7|passage: Role of $18.00 Assistant Manager at McDonald's in Whitevil...|        1|
|   B16|passage: Role of $4,500 Sign on Bonus - MDS Coordinator at The Good...|        1|
|   B18|passage: Role of $45/hr - School RN at Maxim Healthcare Staffing in...|        2|
|   B50|passage: Role of (CAN) Department Manager -FR at Walmart Canada in ...|        1|
|   B58|passage: Role of (CAN) Department Manager at Walmart Canada in Kitc...|        1|
|   B63|passage: Role of (CAN) Department Manager at Walmart Canada in Peac...|        1|
|   B66|passage: Role of (CAN) GT - Senior Software Developer at Walmart Ca...|        2|
|   

In [12]:
# save final output
output_path = os.path.join(PROJECT_ROOT, 'ingest_job_postings', 'output', 'final', 'jobs_to_embed.parquet')
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# save with isco_code for later stratified splitting
final_output.coalesce(1).write.mode('overwrite').parquet(output_path)

print(f'saved {sampled_count:,} jobs to: {output_path}')

saved 165,193 jobs to: /home/developer/project/output/final/jobs_to_embed.parquet


In [13]:
# save full UUID to B ID mapping (for all 1.3M jobs, not just sampled)
mapping_df = jobs_with_ids.select('original_uuid', 'job_id')
mapping_path = os.path.join(PROJECT_ROOT, 'ingest_job_postings', 'output', 'final', 'uuid_to_bid_mapping.parquet')
mapping_df.coalesce(1).write.mode('overwrite').parquet(mapping_path)

print(f'saved full UUID mapping ({total_jobs:,} jobs) to: {mapping_path}')

saved full UUID mapping (1,345,711 jobs) to: /home/developer/project/output/final/uuid_to_bid_mapping.parquet


In [14]:
# save ISCO distribution as JSON for reference
isco_dist = {
    'original_total': total,
    'sampled_total': sampled_count,
    'target_size': TARGET_SAMPLE_SIZE,
    'domains': {}
}

for row in sampled_isco:
    code = row['isco_code']
    count = row['count']
    isco_dist['domains'][str(code)] = {
        'name': ISCO_NAMES.get(code, 'Unknown'),
        'original_count': domain_counts.get(code, 0),
        'sampled_count': count,
        'percentage': round(count / sampled_count * 100, 1)
    }

isco_path = os.path.join(PROJECT_ROOT, 'ingest_job_postings', 'output', 'final', 'isco_distribution.json')
with open(isco_path, 'w') as f:
    json.dump(isco_dist, f, indent=2)

print(f'saved ISCO distribution to: {isco_path}')

saved ISCO distribution to: /home/developer/project/output/final/isco_distribution.json


In [15]:
kafka_config = {
    'bootstrap.servers': KAFKA_BROKER,
    'client.id': 'final-embedding-producer'
}

try:
    producer = Producer(kafka_config)
    print('kafka producer initialized')
    kafka_available = True
except Exception as e:
    print(f'kafka not available: {e}')
    print('skipping kafka publishing')
    kafka_available = False

kafka producer initialized


In [16]:
if kafka_available:
    print('publishing to kafka topic: jobs_to_embed')
    
    # collect to pandas for iteration
    pdf = final_output.select('job_id', 'embedding_text').toPandas()
    
    published = 0
    for idx, row in pdf.iterrows():
        msg = json.dumps({
            'job_id': row['job_id'],
            'embedding_text': row['embedding_text']
        })
        
        producer.produce('jobs_to_embed', value=msg.encode('utf-8'))
        published += 1
        
        if published % 10000 == 0:
            producer.flush()
            print(f' published {published:,} jobs...')
    
    producer.flush()
    print(f'\npublished {published:,} jobs to kafka')
else:
    published = 0
    print('kafka publishing skipped')

publishing to kafka topic: jobs_to_embed
 published 10,000 jobs...
 published 20,000 jobs...
 published 30,000 jobs...
 published 40,000 jobs...
 published 50,000 jobs...
 published 60,000 jobs...
 published 70,000 jobs...
 published 80,000 jobs...
 published 90,000 jobs...
 published 100,000 jobs...
 published 110,000 jobs...
 published 120,000 jobs...
 published 130,000 jobs...
 published 140,000 jobs...
 published 150,000 jobs...
 published 160,000 jobs...

published 165,193 jobs to kafka


In [17]:
from pyspark.sql.functions import avg, min as spark_min, max as spark_max

length_stats = final_output.agg(
    avg(length('embedding_text')).alias('avg_len'),
    spark_min(length('embedding_text')).alias('min_len'),
    spark_max(length('embedding_text')).alias('max_len')
).collect()[0]

print(f'original jobs: {total_jobs:,}')
print(f'sampled jobs: {sampled_count:,}')
print(f'sampling ratio: {sampled_count/total_jobs*100:.1f}%')
print(f'\nISCO domains: {len(sampled_isco)}')
print(f'\nembedding text lengths:')
print(f' average: {length_stats["avg_len"]:.0f} chars')
print(f' min: {length_stats["min_len"]} chars')
print(f' max: {length_stats["max_len"]} chars')

original jobs: 1,345,711
sampled jobs: 165,193
sampling ratio: 12.3%

ISCO domains: 10

embedding text lengths:
 average: 339 chars
 min: 95 chars
 max: 2282 chars


In [18]:
print(f'input: {input_path}')
print(f'\nstratified sampling:')
print(f' original: {total_jobs:,} jobs')
print(f' sampled: {sampled_count:,} jobs')
print(f' ratio: 1:{total_jobs//sampled_count}')
print(f'\noutputs:')
print(f' parquet: {output_path}')
print(f' mapping: {mapping_path}')
print(f' isco: {isco_path}')
if kafka_available:
    print(f' kafka: jobs_to_embed topic ({published:,} messages)')
print(f'\nbalance with CVs:')
print(f'  7K CVs : {sampled_count:,} jobs = 1:{sampled_count//7000}')

input: /home/developer/project/output/unified_job_postings/unified_jobs.parquet

stratified sampling:
 original: 1,345,711 jobs
 sampled: 165,193 jobs
 ratio: 1:8

outputs:
 parquet: /home/developer/project/output/final/jobs_to_embed.parquet
 mapping: /home/developer/project/output/final/uuid_to_bid_mapping.parquet
 isco: /home/developer/project/output/final/isco_distribution.json
 kafka: jobs_to_embed topic (165,193 messages)

balance with CVs:
  7K CVs : 165,193 jobs = 1:23


In [19]:
spark.stop()
print('spark stopped')

spark stopped
